# Text to Speech with Silero

---

[Github](https://github.com/eugenesiow/practical-ml/) | More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml)

---

Notebook to convert an input piece of text into an speech audio file automatically.

[Text-To-Speech synthesis](https://paperswithcode.com/task/text-to-speech-synthesis) is the task of converting written text in natural language to speech.

The [model used](https://pytorch.org/hub/snakers4_silero-models_tts/) is one of the pre-trained `silero_tts` model. It was trained on a private dataset.

Do note that the Silero models are [licensed](https://habr.com/ru/post/549482/) under a [GPU A-GPL 3.0 License](https://github.com/snakers4/silero-models/blob/master/LICENSE) where you have to provide source code if you are using it for commercial purposes.

The notebook is structured as follows:
* Setting up the Environment
* Using the Model (Running Inference)
* Apply Speech Enhancement/Noise Reduction

# Setting up the Environment

#### Dependencies and Runtime

If you're running this notebook in Google Colab, most of the dependencies are already installed and we don't need the GPU for this particular example.

If you decide to run this on many (>thousands) images and want the inference to go faster though, you can select `Runtime` > `Change Runtime Type` from the menubar. Ensure that `GPU` is selected as the `Hardware accelerator`.

We need to install `torchaudio` and `omegaconf` for this example to run, so execute the command below to setup the dependencies.

In [1]:
!pip install -q torchaudio omegaconf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.3 MB/s eta 0:00:00


# Using the Model (Running Inference)

Now we want to load and run the specific Silero 16khz english speaker model. The full set of [available models](https://github.com/snakers4/silero-models#text-to-speech) include models in German and Russian.

Specifically we are running the following steps:

* `torch.hub.load()` - Downloads and loads the pre-trained model from torchhub. In particular, we specify to use the `silero_tts` model with the `en` (English) language speaker `lj_16khz`.
* `model.to(device)` - We load the model to the `CPU` (the default) or `GPU` (if you set this up in the previous section) for inferencing.

In [55]:
import torch

device = torch.device('cpu')
torch.set_num_threads(4)
speaker = 'en_102'  # 'aidar', 'baya', 'kseniya', 'xenia', 'random'
sample_rate = 8000 # 8000, 24000, 48000

model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language='en',
                                     speaker='v3_en')
model.to(device)
audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate)

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


In [47]:
import torch
device = torch.device('cpu')
torch.set_num_threads(4)
language = 'en'
speaker = 'en_81'
sample_rate = 8000
device = torch.device('cpu')
model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language='en',
                                     speaker='v3_en')
model.to(device)
audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate)


Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


Now we define the `example_text` variable, a piece of text that we want to convert to a speech audio file. Next, we synthesize/generate the audio file.

The notebook will then display the audio sample produced for us to playback.

In [48]:
from IPython.display import Audio, display

example_text = 'What is umbrage? According to the Oxford Languages dictionary, Umbrage is a noun that means offence or annoyance.'
audio = model.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate)

# Convert the PyTorch tensor to a NumPy array and squeeze to 1D
audio_np = audio.cpu().numpy().squeeze()

display(Audio(audio_np, rate=sample_rate))

We notice that there is some noise in the generated sample which can easily be reduced to enhance the quality of speech using a speech enhancement model. We try this in the next section. This is entirely optional.

# Apply Speech Enhancement/Noise Reduction

We use the simple and convenient LogMMSE algorithm (Log Minimum Mean Square Error) with the [logmmse library](https://github.com/wilsonchingg/logmmse).

In [56]:
!pip install -q logmmse

Run the LogMMSE algorithm on the generated audio `audio[0]` and  display the enhanced audio sample produced in an audio player.

In [73]:
print(f"Shape of audio_np: {audio_np.shape}")


Shape of audio_np: ()


In [74]:
from logmmse import logmmse
from IPython.display import Audio, display

# Apply noise reduction
enhanced_audio = logmmse(audio_np, sample_rate)

# Play the audio
display(Audio(enhanced_audio, rate=sample_rate))


TypeError: iteration over a 0-d array

In [70]:
import numpy as np
from logmmse import logmmse

enhanced = logmmse(np.array(audio[0]), sample_rate, output_file=None, initial_noise=1, window_size=160, noise_threshold=0.15)
display(Audio(enhanced, rate=sample_rate))

TypeError: iteration over a 0-d array

Save the enhanced audio to file.

In [ ]:
from scipy.io.wavfile import write

write('/content/audio.wav', sample_rate, enhanced)

We can connect to Google Drive with the following code. You can also click the `Files` icon on the left panel and click `Mount Drive` to mount your Google Drive.

The root of your Google Drive will be mounted to `/content/drive/My Drive/`. If you have problems mounting the drive, you can check out this [tutorial](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

You can move the output files which are saved in the `/content/` directory to the root of your Google Drive.

In [ ]:
import shutil
shutil.move('/content/audio.wav', '/content/drive/My Drive/audio.wav')

More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml) and do star or drop us some feedback on how to improve the notebooks on the [Github repo](https://github.com/eugenesiow/practical-ml/).